In [12]:
!pip install faiss-cpu tensorflow transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.8 MB ? eta -:--:--
    --------------------------------------- 0.3/13.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.8 MB 882.6 kB/s eta 0:00:16
   - -------------------------------------- 0.5/13.8 MB 882.6 kB/s eta 0:00:16
   -- ------------------------------------- 0.8/13.8 MB 799.2 kB/s eta 0:00:17
   -- ------------------------------------- 0.8/13.8 MB 799.2 kB/s eta 0:00:17
   --- ------------------------------------ 1.0/13.8 MB 825.2 kB/s eta 0:00:16
   --- ------------------------------------ 1.3/13.8 MB 828.9 kB/s eta 0:00:16
   --- ------------------------------------ 1.3/13.8 MB 828.9 kB/s eta 0:00:16
   ---- ----------------------------------- 1.6/13.8 MB 791.5 kB/s eta 0:00:16
   ----- ---------------------------------- 1.8/13.8 MB 799.2 kB/s eta 0:00:16
   ---

In [4]:
!pip install pillow tf-keras

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 4.5 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import tensorflow as tf
import faiss
import os 
from tqdm import tqdm
from transformers import ViTImageProcessor, TFViTModel
from PIL import Image  

c:\Users\mbouchoucha\Desktop\test\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Path to your image
image_path = ".\data\image.png"  # Replace with your image path

feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = TFViTModel.from_pretrained('google/vit-base-patch16-224')

def featuregetter(image_path,feature_extractor=feature_extractor,model=model):
    # Load the image using PIL
    image = Image.open(image_path).convert("RGB")  # Ensure the image is in RGB format

    inputs = feature_extractor(images=image, return_tensors="tf")
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state[:,0,:]
    normalized_features = tf.linalg.l2_normalize(last_hidden_states, axis=-1)
    return normalized_features

# print("Predicted class:", last_hidden_states.shape)
featuregetter(image_path).shape

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFViTModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing TFViTModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFViTModel were not initialized from the PyTorch model and are newly initialized: ['vit.pooler.dense.weight', 'vit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TensorShape([1, 768])

zid partie win twalli te5dem batch processing !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [23]:
index = faiss.IndexFlatL2(featuregetter(image_path).shape[1])
l=[os.path.join('data',i) for i in  os.listdir("data") if i.endswith(".webp") or i.endswith(".png") or i.endswith(".jpg") or i.endswith(".jpeg")]
print(l)
for i in tqdm(l):
    index.add(featuregetter(i).numpy())

['data\\2023-12-04_Orange_cat_behavior_Hero.jpg', 'data\\amur-tiger-01-01.webp', 'data\\image.png', 'data\\IMG_9210-1024x683.webp', 'data\\téléchargé.jpg', 'data\\Walking_tiger_female.jpg']


100%|██████████| 6/6 [00:01<00:00,  3.54it/s]


In [24]:
import math

number_of_clusters = 4*math.ceil(math.sqrt(768))

quantizer = faiss.IndexFlatL2(768)

index = faiss.IndexIVFFlat(quantizer, 768, 2, faiss.METRIC_L2)

In [25]:
features = np.stack([featuregetter(img)[0] for img in l])
index.train(features)

In [26]:
index.add(features)

In [27]:
test=featuregetter("Front Page 1.2025.jpg").numpy()
k = 4
index.nprobe = 10  # Number of clusters to explore
D, I = index.search(test, k)

print("Indices of nearest neighbors:")
print(I)
print("\nDistances to nearest neighbors:")
print(D)

Indices of nearest neighbors:
[[0 3 1 5]]

Distances to nearest neighbors:
[[1.0935392 1.3429668 1.3610501 1.600475 ]]


In [28]:
for i, neighbors in enumerate(I):
    print(f"Query {i + 1}:")
    for idx,neighbor_idx in enumerate(neighbors):
        print(f"  Nearest neighbor index: {neighbor_idx}, File path: {l[neighbor_idx]}, Distance: {D[0][idx]}")

Query 1:
  Nearest neighbor index: 0, File path: data\2023-12-04_Orange_cat_behavior_Hero.jpg, Distance: 1.0935392379760742
  Nearest neighbor index: 3, File path: data\IMG_9210-1024x683.webp, Distance: 1.3429667949676514
  Nearest neighbor index: 1, File path: data\amur-tiger-01-01.webp, Distance: 1.3610501289367676
  Nearest neighbor index: 5, File path: data\Walking_tiger_female.jpg, Distance: 1.6004749536514282


possible classes in this link : 
https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/